In [7]:
import pymongo
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams, Distance
from transformers import AutoTokenizer, AutoModel
import numpy as np

# MongoDB client setup
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
db = mongo_client["rag_system"]
collection = db["featurized_data"]

from qdrant_client.models import VectorParams, Distance

# Qdrant client setup
qdrant_client = QdrantClient("http://localhost:6333")
qdrant_client.recreate_collection(
    collection_name="vectors",
    vectors_config=VectorParams(
        size=768,
        distance=Distance.COSINE,
    ),
)

# Featurization Pipeline
def featurize_and_store(data, model_name="bert-base-uncased"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)

    for doc_id, text in enumerate(data):
        tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
        embeddings = model(**tokens).last_hidden_state.mean(dim=1).detach().numpy()
        
        # Store in MongoDB
        collection.insert_one({"_id": doc_id, "text": text, "embedding": embeddings.tolist()})
        
        # Store in Qdrant
        qdrant_client.upsert(
            collection_name="vectors",
            points=[PointStruct(id=doc_id, vector=embeddings[0].tolist(), payload={"text": text})]
        )

data = ["Sample text 1", "Sample text 2"]  # Replace with your ROS-related data
featurize_and_store(data)


/var/folders/qs/5tb486nj6bv4jc33tdzjs8qr0000gn/T/ipykernel_2918/1792487890.py:16: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


ResponseHandlingException: [Errno 61] Connection refused

In [ ]:
import gradio as gr
from qdrant_client import QdrantClient
from transformers import pipeline

# Load Retrieval Augmented Generation pipeline
rag_pipeline = pipeline("question-answering", model="facebook/dpr-reader-single-nq-base")

# Qdrant setup
qdrant_client = QdrantClient("http://localhost:6333")

def answer_question(question):
    # Retrieve relevant data
    search_results = qdrant_client.search(
        collection_name="vectors",
        query_vector=[0.1] * 768,  # Replace with featurized question
        limit=1
    )
    retrieved_text = search_results[0].payload["text"]
    
    # Generate answer
    response = rag_pipeline({"question": question, "context": retrieved_text})
    return response["answer"]

# Gradio App
def run_app():
    demo = gr.Interface(
        fn=answer_question,
        inputs=gr.Textbox(label="Ask a question"),
        outputs=gr.Textbox(label="Answer"),
        examples=[
            ["Tell me how can I navigate to a specific pose - include replanning aspects in your answer."],
            ["Can you provide me with code for this task?"]
        ]
    )
    demo.launch()

run_app()
